In [ ]:
from bs4 import BeautifulSoup
#import requests
from urllib.request import Request, urlopen
import numpy as np
import time
import re
import os

In [ ]:
SITENAME = "metacritic"
SITEMAP_SEED = 'http://www.metacritic.com/sitemap/Video-game/3/sitemap.xml'
PAGES_SAVE_DIR = "pages-metacritic"
TABLES_SAVE_DIR = "tables-metacritic"
TEMPLATE_DEEP = 3
IGNORE_LAST_NTH = 1

In [ ]:
#r = requests.get(SITEMAP_SEED) #URL della sitemap del sito target
#xml = r.text
xml = open(".\\sitemapmeta.xml").read()
soup = BeautifulSoup(xml)
sitemapTags = soup.find_all("loc")
pages = np.array([])

for page in sitemapTags: 
    pages  = np.append(pages, page.string)
    


In [ ]:
pages

In [ ]:
import pandas as pd
pd.options.display.max_colwidth = 100


df = pd.DataFrame(data=pages)
df = df.loc[df[0].str.contains("http://www\..+\.\w+(/[^/]+){"+str((TEMPLATE_DEEP+2))+"}$" , regex = True)]

df

In [ ]:
page_dir = ".\\"+PAGES_SAVE_DIR+"\\"
if not os.path.exists(page_dir):
    os.makedirs(page_dir)
    
t_page_dir = ".\\"+TABLES_SAVE_DIR+"\\"
if not os.path.exists(t_page_dir):
    os.makedirs(t_page_dir)

In [ ]:
index = 0
page_ref_table = []
page_analized = []
for page in df[0]:    
    page = re.sub("(/[^/]+){"+str(IGNORE_LAST_NTH)+"}$" , "" , page)
    for p in range(0,TEMPLATE_DEEP):
        page_template = re.sub("(/[^/]+){"+str(p)+"}$" , "" , page)
        if page_template not in page_analized :
            req = Request(page_template, headers={'User-Agent': 'Mozilla/5.0'})
            web_byte = urlopen(req).read()
            html = web_byte.decode('utf-8')
            #resp = requests.get(page_template)
            #html = resp.content.decode("utf-8")
            filepath = page_dir+str(index)+'.html'
            f = open(filepath, 'w' , encoding='utf-8')
            f.write(html)
            f.close
            time.sleep(0.5) #intervallo di tempo per non incorrere in filtri anti DDOS
            page_ref_table.append([page_template , p , filepath])
            page_analized.append(page_template)
            print("saved "+filepath)
            print()
            index += 1

In [ ]:
pages_array = np.array(page_ref_table)    
np.save(t_page_dir+"pages.npy" , pages_array)

In [ ]:
a = np.load(t_page_dir+"pages.npy")